## Getting API NEWS


In [ ]:
# downloading libraries
import requests
from newspaper import Article
import lxml.html.clean
import nltk
nltk.download('punkt_tab')
import pandas as pd
from datetime import datetime

In [ ]:
def get_news(query):
  api_key = 'pub_7364518abd44eff6ae1cb8bf439fb91cdce85'
  query = query
  url = f'https://newsdata.io/api/1/news?apikey={api_key}&q={query}&language=en'
  response = requests.get(url)
  data = response.json()
  print(f"Request returned {response.status_code} : '{response.reason}'")
  article_data = []
  if data and data.get('status') == 'success':
    articles = data.get('results', [])
    for article in articles:
        title = article.get('title')
        url = article.get('link')
        content, summary = None, None

        if url:
            try:
                news_article = Article(url)
                news_article.download()
                news_article.parse()
                content = news_article.text

                news_article.nlp()
                summary = news_article.summary
            except Exception as e:
                print(f"Failed to scrape {url}: {e}")

        # adding article information to the list
            article_data.append({
            "Title": title,
            "URL": url,
            "Content": content,
            "Summary": summary

        })
        else:
            print(f"No URL found for article titled: {title}")
  else:
      print("No articles found for the given query.")
  return pd.DataFrame(article_data)

In [ ]:
get_news('Latest News')

In [ ]:
api_key = 'pub_7364518abd44eff6ae1cb8bf439fb91cdce85'
query = 'Latest News'
url = f'https://newsdata.io/api/1/news?apikey={api_key}&q={query}'

response = requests.get(url)
data = response.json()

print(f"Request returned {response.status_code} : '{response.reason}'")



In [ ]:
# empty list to load all articles into
article_data = []

# scrapping article data
if data and data.get('status') == 'success':
    articles = data.get('results', [])
    for article in articles:
        title = article.get('title')
        url = article.get('link')
        content, summary = None, None

        if url:
            try:
                news_article = Article(url)
                news_article.download()
                news_article.parse()
                content = news_article.text


                news_article.nlp()
                summary = news_article.summary
            except Exception as e:
                print(f"Failed to scrape {url}: {e}")

        # adding article information to the list
            article_data.append({
            "Title": title,
            "URL": url,
            "Content": content,
            "Summary":summary

        })
        else:
            print(f"No URL found for article titled: {title}")
else:
    print("No articles found for the given query.")

In [ ]:
df = get_news('Latest News')

In [ ]:
import cohere

co = cohere.Client('OqqyzITiJdIuS4hVpIJT7V70qcknzNGn2EAbias1')

def generate_embedding(content):
    if content:
        response = co.embed(texts=[content], model='embed-english-v2.0')
        return response.embeddings[0]
    return None

def cohere_prompt(prompt, model='command', max_tokens=300):
    response = co.generate(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens
    )
    return response.generations[0].text.strip()

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')

In [ ]:
def summarize_text(content):
    if content:
        input_text = f"summarize: {content}"  # T5 expects a 'summarize:' prefix
        inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)
        summary_ids = model.generate(inputs.input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return None

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def extractive_summary(text, sentence_count=4):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)

    return " ".join(str(sentence) for sentence in summary)

Source: https://www.geeksforgeeks.org/mastering-text-summarization-with-sumy-a-python-library-overview/

## Function for Getting News Content

In [ ]:
def get_news(query):
  api_key = 'pub_7364518abd44eff6ae1cb8bf439fb91cdce85'
  query = 'Latest News'
  url = f'https://newsdata.io/api/1/news?apikey={api_key}&q={query}&language=en'
  response = requests.get(url)
  data = response.json()
  print(f"Request returned {response.status_code} : '{response.reason}'")
  article_data = []
  if data and data.get('status') == 'success':
    articles = data.get('results', [])
    for i, article in enumerate(articles):
        title = article.get('title')
        url = article.get('link')
        content, summary = None, None

        if url:
            try:
                news_article = Article(url)
                news_article.download()
                news_article.parse()
                content = news_article.text

                news_article.nlp()
                summary = extractive_summary(content)
            except Exception as e:
                print(f"Failed to scrape {url}: {e}")

        # adding article information to the list
            article_data.append({
            "Title": title,
            "URL": url,
            "Content": content,
            "Extractive Summary": summary,
            "Abstractive Summary": summarize_text(content),
            "Embedding": generate_embedding(content)
        })
        else:
            print(f"No URL found for article titled: {title}")
  else:
      print("No articles found for the given query.")
  return pd.DataFrame(article_data)

In [ ]:
df = get_news('Latest News')

In [ ]:
df

## Similarity Search

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_similar_articles(query, df, k=3):
    query_embedding = np.array(generate_embedding(query)).reshape((1, -1))
    df_filtered = df[df['Embedding'].notnull()].copy()
    embeddings = np.stack(df_filtered['Embedding'].values)
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    df_filtered['Similarity'] = similarities
    top_k = df_filtered.sort_values('Similarity', ascending=False).head(k)
    return top_k

In [ ]:
query = "federal"
top_k_articles = get_top_k_similar_articles(query, df, k=1000000)

In [ ]:
top_k_articles

## Final Summary

In [ ]:
def generate_summaries(query, df, k=5):
    # Get top-k articles
    top_k_articles = get_top_k_similar_articles(query, df, k)

    # Extractive summaries
    extractive_summaries = "ARTICLE: " + "\n\nARTICLE: ".join(top_k_articles['Extractive Summary'].values)
    extractive_prompt = f"Summarize the following articles into a cohesive extractive summary in under 200 words with proper flow. It should have something from each article:\n{extractive_summaries}"

    # Abstractive summaries
    abstractive_summaries = "ARTICLE: " + "\n\nARTICLE: ".join(top_k_articles['Abstractive Summary'].values)
    abstractive_prompt = f"Summarize the following articles into a cohesive abstractive summary in under 200 words with proper flow. It should have something from each article:\n{abstractive_summaries}"

    print("EXTRACTIVE PROMPT:\n", extractive_prompt)
    extractive_summary = cohere_prompt(extractive_prompt)

    print("\n\nABSTRACTIVE PROMPT:\n", abstractive_prompt)
    abstractive_summary = cohere_prompt(abstractive_prompt)

    return extractive_summary, abstractive_summary

In [ ]:
extractive, abstractive = generate_summaries('bollywood', df, k=5)

In [ ]:
extractive

In [ ]:
abstractive

## Model Evaluation

In [ ]:
from rouge_score import rouge_scorer
import numpy as np

In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

In [ ]:
def evaluate_summaries_with_rouge(df):
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for idx, row in df.iterrows():
        reference = row['Summary']
        generated = row['Content_Summary']

        if pd.notnull(reference) and pd.notnull(generated):
            scores = scorer.score(reference, generated)
            rouge1_scores.append(scores["rouge1"].fmeasure)
            rouge2_scores.append(scores["rouge2"].fmeasure)
            rougeL_scores.append(scores["rougeL"].fmeasure)

    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)

    return {
        "ROUGE-1": round(avg_rouge1, 4),
        "ROUGE-2": round(avg_rouge2, 4),
        "ROUGE-L": round(avg_rougeL, 4)
    }

In [ ]:
results = evaluate_summaries_with_rouge(df)
print(results)

Source: https://medium.com/nlplanet/two-minutes-nlp-learn-the-rouge-metric-by-examples-f179cc285499

# Database in AWS

## Loading Hugging Face Data

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import hf_hub_download
import pandas as pd

# huggingface repo id
repo_id = "abisee/cnn_dailymail"


filenames = [
    "3.0.0/test-00000-of-00001.parquet",
    "3.0.0/train-00000-of-00003.parquet",
    "3.0.0/train-00001-of-00003.parquet",
    "3.0.0/train-00002-of-00003.parquet",
    "3.0.0/validation-00000-of-00001.parquet"
]

dfs = {
    filename: pd.read_parquet(hf_hub_download(
        repo_id=repo_id, filename=filename, repo_type="dataset"
    ))
    for filename in filenames
}


test_df = dfs["3.0.0/test-00000-of-00001.parquet"]
train_df = pd.concat([dfs[f] for f in filenames if "train" in f], ignore_index=True)
val_df = dfs["3.0.0/validation-00000-of-00001.parquet"]


## Storing to S3

In [ ]:
test_df

In [ ]:
#save file to parquet
test_df.to_parquet('test.parquet')
train_df.to_parquet('train.parquet')
val_df.to_parquet('val.parquet')

In [ ]:
import boto3
def upload_to_s3(file_path, bucket_name, s3_key):
  s3 = boto3.client('s3')
  try:
    s3.upload_file(file_path, bucket_name, s3_key)
    print(f"File '{file_path}' successfully uploaded to 's3://{bucket_name}/{s3_key}'")
  except Exception as e:
    print(f"Error uploading file: {e}")

In [ ]:
!jupyter nbconvert --clear-output --inplace NewsSumme.ipynb

In [ ]:
!jupyter nbconvert NewsSumme.ipynb \
  --ClearMetadataPreprocessor.enabled=True \
  --inplace

In [ ]:
import nbformat

nb = nbformat.read("NewsSumme.ipynb", as_version=4)

# Remove broken widgets metadata
nb.metadata.pop("widgets", None)

nbformat.write(nb, "NewsSumme.ipynb")
